In [1]:
!pip install -q transformers datasets evaluate accelerate torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("ag_news")
print(dataset)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/190 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [4]:
def preprocess(example):
    text = f"classify: {example['text']}"
    label = str(example["label"])

    inputs = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128
    )

    labels = tokenizer(
        label,
        truncation=True,
        padding="max_length",
        max_length=10
    )

    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized = dataset.map(preprocess)


Map:   0%|          | 0/120000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=1,
)


In [7]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"]
        .shuffle(seed=42)
        .select(range(5000)),
    eval_dataset=tokenized["test"]
        .shuffle(seed=42)
        .select(range(1000)),
)

trainer.train()


Step,Training Loss
500,8.721562
1000,6.531153


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=1250, training_loss=7.34971923828125, metrics={'train_runtime': 189.8272, 'train_samples_per_second': 52.679, 'train_steps_per_second': 6.585, 'total_flos': 464726261760000.0, 'train_loss': 7.34971923828125, 'epoch': 2.0})

In [8]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 5.720645904541016, 'eval_runtime': 8.6527, 'eval_samples_per_second': 115.571, 'eval_steps_per_second': 14.446, 'epoch': 2.0}


In [10]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

text = "NASA launched a new satellite today."

inputs = tokenizer(
    "classify: " + text,
    return_tensors="pt"
).to(device)

outputs = model.generate(**inputs)

print(
    "Predicted label:",
    tokenizer.decode(outputs[0], skip_special_tokens=True)
)


Predicted label: 3


 Introduction
 Dataset Description
 Model Description
 Training Process
 Evaluation
 Observations
 Conclusion